In [1]:
from caffe import layers as L, params as P
import caffe

In [2]:
caffe.set_mode_gpu()
caffe.set_device(1)

In [3]:
def lenet(data_set, batch_size, scope):
    n = caffe.NetSpec()
    n.data, n.label = L.ImageData(batch_size=batch_size, source=data_set, is_color=False, ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, pad=3, num_output=32, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=3, num_output=64, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv3 = L.Convolution(n.pool2, kernel_size=3, num_output=128, weight_filler=dict(type='xavier'))
    n.pool3 = L.Pooling(n.conv3, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv4 = L.Convolution(n.pool3, kernel_size=3, num_output=256, weight_filler=dict(type='xavier'))
    n.pool4 = L.Pooling(n.conv4, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv5 = L.Convolution(n.pool4, kernel_size=3, num_output=512, weight_filler=dict(type='xavier'))
    n.pool5 = L.Pooling(n.conv5, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.fc1 =   L.InnerProduct(n.pool5, num_output=1000, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.fc1, in_place=True)
    n.fc2 = L.InnerProduct(n.relu1, num_output=500, weight_filler=dict(type='xavier'))
    n.relu2 = L.ReLU(n.fc2, in_place=True)
    n.fc3 = L.InnerProduct(n.relu2, num_output=200, weight_filler=dict(type='xavier'))
    n.relu3 = L.ReLU(n.fc3, in_place=True)
    n.score = L.InnerProduct(n.relu2, num_output=10, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    if scope == 'TEST':
        n.accuracy_top1 = L.Accuracy(n.score, n.label)
        n.accuracy_top5 = L.Accuracy(n.score, n.label, accuracy_param=dict(top_k=5))
        
    return n.to_proto()
    
with open('proto/lenet_auto_train.prototxt', 'w') as f:
    f.write(str(lenet('resources/train.txt', 256, 'TRAIN')))
    
with open('proto/lenet_auto_test.prototxt', 'w') as f:
    f.write(str(lenet('resources/test.txt', 256, 'TEST')))

In [4]:
solver = caffe.RMSPropSolver('proto/lenet_solver.prototxt')
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

[('data', (256, 1, 224, 224)),
 ('label', (256,)),
 ('conv1', (256, 32, 226, 226)),
 ('pool1', (256, 32, 113, 113)),
 ('conv2', (256, 64, 111, 111)),
 ('pool2', (256, 64, 56, 56)),
 ('conv3', (256, 128, 54, 54)),
 ('pool3', (256, 128, 27, 27)),
 ('conv4', (256, 256, 25, 25)),
 ('pool4', (256, 256, 13, 13)),
 ('conv5', (256, 512, 11, 11)),
 ('pool5', (256, 512, 6, 6)),
 ('fc1', (256, 1000)),
 ('fc2', (256, 500)),
 ('fc2_relu2_0_split_0', (256, 500)),
 ('fc2_relu2_0_split_1', (256, 500)),
 ('fc3', (256, 200)),
 ('score', (256, 10)),
 ('loss', ())]

In [4]:
solver.solve()

NameError: name 'solver' is not defined